<a href="https://colab.research.google.com/github/AditiLORenzo/Aditiprojects/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data=pd.read_excel("/content/insurance 2.xlsx")
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

Feature correlation

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))

corr=data.corr()
sns.heatmap(corr,annot=True,ax=ax)

In [ ]:
import pickle
def save_object(obj , name):
    pickle_obj = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_obj)
    pickle_obj.close()

Label encode Object Type

In [ ]:
d_types=dict(data.dtypes)
for name,type_ in d_types.items(): 
   if str(type_)=='object':
     print(f"<===={name}===>")
     print(data[name].value_counts())
     print()

In [ ]:
dict(data['region'].value_counts())

In [ ]:
from sklearn.preprocessing import LabelEncoder
for name,type_ in d_types.items():
   if str(type_)=='object':
      le=LabelEncoder()
      data[name]=le.fit_transform(data[name])
      save_object(le , f"Label_Encoder_{name}")

Check info after label encoding

In [ ]:
data.info()

One hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder=OneHotEncoder()

In [ ]:
part=onehotencoder.fit_transform(data['region'].values.reshape(-1,1)).toarray()
value=dict(data['region'].value_counts())

In [ ]:
print(part)

In [ ]:
value.items()

In [ ]:
for gig in enumerate(value.items()):
   print(gig)
  # print(e)

In [ ]:
for e , (val , _) in enumerate(value.items()):
    data["region_" + str(val)] = part[:,e]
  # print(_)


In [ ]:
data=data.drop(["region"],axis=1)

In [ ]:
data.head()

Handle skewness in predictive column

In [ ]:
orginal_y=data['expenses'].values.copy()

In [ ]:
orginal_y

In [ ]:
print("print data skewness :", data['expenses'].skew())
plt.hist(data['expenses'])
plt.show()

In [ ]:
col_log=np.log(data["expenses"])
print("Skewness in column: Log Expenses",col_log.skew())
plt.hist(col_log)
plt.show()

In [ ]:
col_sqrt=np.sqrt(data["expenses"])
print("skewnes in column:sqrt expenses",col_sqrt.skew())
plt.hist(col_sqrt)
plt.show()

In [ ]:
from scipy import stats
col_cox,lam=stats.boxcox(data['expenses'])[0:2]
save_object(lam,"boxcox_lambda")
plt.hist(col_cox)
plt.show()

Make features and target

In [ ]:
remaining_columns=list(data.columns)
remaining_columns.remove("expenses")

In [ ]:
save_object(remaining_columns , "columns")

In [ ]:
X=data[remaining_columns].values
Y=data['expenses'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.4,random_state=4)
scaler=StandardScaler()



In [ ]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
save_object(scaler , "Scaler")

In [ ]:
#check wheather data is standarized or not
plt.ylim(-1,1)
mean=[]
for i in range(X_train.shape[1]):
    mean.append(np.mean(X_train[:,i]))
plt.plot(mean,scaley=False)
     #(np.mean(X_train[:,i]))

In [ ]:
#check variance
plt.ylim(0,2)
var=[]
for i in range(X_train.shape[1]):
    var.append(np.var(X_train[:,i]))
plt.plot(var)

In [ ]:
np.mean
   # plt.plot(mean,scaley=False)

In [ ]:
#col_cox

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=7)
X_train=pca.fit_transform(X_train)
X_test=pca.transform(X_test)

In [ ]:
pca.explained_variance_ratio_.cumsum()
save_object(pca , "PCA")

DEFINE METRICS

In [ ]:
def rmse_score(y_test , y_pred):
    value = (1/len(y_test))*np.sum((y_test - y_pred)**2)
    return np.sqrt(value)

def r2_score(y_test , y_pred):
    numenator = (1/len(y_test))*np.sum((y_test - y_pred)**2)
    denominator = (1/len(y_test))*np.sum((y_test - np.mean(y_test))**2)
    return (1 - (numenator/denominator))

def mae(y_test , y_pred):
    return (1/len(y_test))*np.sum(np.abs(y_test - y_pred))

def adj_r2_score(y_test , y_pred , n_features):
    numenator = (1-r2_score(y_test , y_pred))*(len(y_test) - 1)
    denominator = len(y_test) - n_features - 1
    return 1 - (numenator/denominator)

In [ ]:
model=LinearRegression()
model.fit(X_train,Y_train)
ypred=model.predict(X_test)

In [ ]:
print("LR model score",model.score(X_train,Y_train))
print("LR model score",model.score(X_test,Y_test))

In [ ]:
rmse_score(Y_test,ypred)

In [ ]:
r2_score(Y_test,ypred)

In [ ]:
mae(Y_test,ypred)

##Regularization

In [ ]:
model1=Ridge()
model1.fit(X_train,Y_train)

In [ ]:
#training and testing accuracy
print("Training accuracy:",model1.score(X_train,Y_train))
print("Testing accuracy:",model1.score(X_test,Y_test))


In [ ]:
#Lasso
model2=Lasso()
model2.fit(X_train,Y_train)
print("Training accuracy",model2.score(X_train,Y_train))
print("Testin accuracy",model2.score(X_test,Y_test))

In [ ]:
#Elastic net
model3=ElasticNet()
model3.fit(X_train,Y_train)
print("Training accuracy",model3.score(X_train,Y_train))
print("Testin accuracy",model3.score(X_test,Y_test))


###changing Parameter for elastic and Lasso

In [ ]:
model4=Lasso(alpha=0.02)
model4.fit(X_train,Y_train)
print("Training accuracy",model4.score(X_train,Y_train))
print("Testin accuracy",model4.score(X_test,Y_test))